# HR Analytics

<img src = 'https://datahack-prod.s3.ap-south-1.amazonaws.com/__sized__/contest_cover/hr_1920x480_s5WuoZs-thumbnail-1200x1200-90.jpg'>

Practice Problem: https://datahack.analyticsvidhya.com/contest/wns-analytics-hackathon-2018-1/

## HR Analytics

HR analytics is revolutionising the way human resources departments operate, leading to higher efficiency and better results overall. Human resources has been using analytics for years. However, the collection, processing and analysis of data has been largely manual, and given the nature of human resources dynamics and HR KPIs, the approach has been constraining HR. Therefore, it is surprising that HR departments woke up to the utility of machine learning so late in the game. Here is an opportunity to try predictive analytics in identifying the employees most likely to get promoted.

## Problem Statement

Your client is a large MNC and they have 9 broad verticals across the organisation. One of the problem your client is facing is around identifying the right people for promotion *(only for manager position and below)* and prepare them in time. Currently the process, they are following is:

* They first identify a set of employees based on recommendations/ past performance
* Selected employees go through the separate training and evaluation program for each vertical. These programs are based on the required skill of each vertical
* At the end of the program, based on various factors such as training performance, KPI completion (only employees with KPIs completed greater than 60% are considered) etc., employee gets promotion

For above mentioned process, the final promotions are only announced after the evaluation and this leads to delay in transition to their new roles. Hence, company needs your help in identifying the eligible candidates at a particular checkpoint so that they can expedite the entire promotion cycle. 

<img src = 'https://s3-ap-south-1.amazonaws.com/av-blog-media/wp-content/uploads/2018/09/wns_hack_im_1.jpg'>

They have provided multiple attributes around Employee's past and current performance along with demographics. Now, The task is to predict whether a potential promotee at checkpoint in the test set will be promoted or not after the evaluation process.

## Evaluation Metric

The evaluation metric for this competition is F1 Score.

## Public and Private Split

Test data is further randomly divided into Public (40%) and Private (60%) data.

Your initial responses will be checked and scored on the Public data.
The final rankings would be based on your private score which will be published once the competition is over.

## Entorno

In [1]:
import sys
sys.version

'3.6.12 |Anaconda, Inc.| (default, Sep  8 2020, 17:50:39) \n[GCC Clang 10.0.0 ]'

In [2]:
!conda info --envs

# conda environments:
#
micromaster              /Users/manuel/.conda/envs/micromaster
                         /Users/manuel/.julia/conda/3
base                  *  /Users/manuel/opt/anaconda3
belcorp                  /Users/manuel/opt/anaconda3/envs/belcorp
courseragcp              /Users/manuel/opt/anaconda3/envs/courseragcp
iapucp                   /Users/manuel/opt/anaconda3/envs/iapucp
mitxpro                  /Users/manuel/opt/anaconda3/envs/mitxpro
style-transfer           /Users/manuel/opt/anaconda3/envs/style-transfer
taller-dmc               /Users/manuel/opt/anaconda3/envs/taller-dmc
udacity                  /Users/manuel/opt/anaconda3/envs/udacity



## Paquetes

In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import os
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm, tqdm_notebook
from pathlib import Path
import random
import warnings
import pickle

warnings.filterwarnings('ignore')


seed = 2020
random.seed(seed)

pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 400)
sns.set()

DATA = Path('../../data') 
RAW  = DATA/'raw'
PROCESSED = DATA/'processed'
SUBMISSIONS = DATA/'submissions'    

MODEL = Path('../../model') 

In [4]:
pd.__version__

'1.1.3'

In [5]:
np.__version__

'1.19.1'

In [6]:
sklearn.__version__

'0.23.2'

In [7]:
id_columns = 'employee_id'
target = 'is_promoted'

## Lectura de datos

In [8]:
os.listdir(f'{PROCESSED}')

['preprocess_v2_smoteTomek20_train.csv',
 'preprocess_v2_knnimputation.pkl',
 'preprocess_v1_smote20_train.csv',
 'preprocess_v2_smote50_train.csv',
 'preprocess_v2_over50_train.csv',
 'preprocess_v2_ohe.pkl',
 'preprocess_v2_ohe_columns.pkl',
 '.DS_Store',
 'preprocess_v1_smoteTomek20_train.csv',
 'preprocess_v2_scalerimputation.pkl',
 'preprocess_v1_ohe.pkl',
 'preprocess_v1_smoteTomek50_train.csv',
 'preprocess_v2_scaler.pkl',
 'preprocess_v1_train.csv',
 'preprocess_v2_smote20_train.csv',
 'preprocess_v1_smote50_train.csv',
 'preprocess_v2_smoteTomek50_train.csv',
 'preprocess_v1_ohe_columns.pkl',
 'preprocess_v2_under50_train.csv',
 'preprocess_v2_train.csv',
 'preprocess_v1_scaler.pkl',
 'preprocess_v2_val.csv',
 '.ipynb_checkpoints',
 'preprocess_v1_impute_values.pkl',
 'preprocess_v1_over50_train.csv',
 'preprocess_v1_capping_values.pkl',
 'preprocess_v1_val.csv',
 'preprocess_v1_under50_train.csv',
 'preprocess_v2_capping_values.pkl']

## Entrenamiento V1 sin balanceo

In [9]:
import xgboost as xgb
from sklearn.metrics import precision_recall_curve, roc_auc_score, f1_score

In [10]:
preproc_train = [file for file in os.listdir(f'{PROCESSED}') if file.endswith('train.csv')]
preproc_train

['preprocess_v2_smoteTomek20_train.csv',
 'preprocess_v1_smote20_train.csv',
 'preprocess_v2_smote50_train.csv',
 'preprocess_v2_over50_train.csv',
 'preprocess_v1_smoteTomek20_train.csv',
 'preprocess_v1_smoteTomek50_train.csv',
 'preprocess_v1_train.csv',
 'preprocess_v2_smote20_train.csv',
 'preprocess_v1_smote50_train.csv',
 'preprocess_v2_smoteTomek50_train.csv',
 'preprocess_v2_under50_train.csv',
 'preprocess_v2_train.csv',
 'preprocess_v1_over50_train.csv',
 'preprocess_v1_under50_train.csv']

In [11]:
preproc_val = [file for file in os.listdir(f'{PROCESSED}') if file.endswith('val.csv')]
preproc_val

['preprocess_v2_val.csv', 'preprocess_v1_val.csv']

In [12]:
for train_file in sorted(preproc_train):
    df_train = pd.read_csv(f'{PROCESSED}/{train_file}', compression = 'zip')
    df_val = pd.read_csv(f'{PROCESSED}/{preproc_val[0]}', compression = 'zip')
    
    print(f'label: {train_file:35} \tnrows: {len(df_train)} \t%target train: {df_train[target].mean():.4f} \t%target val: {df_val[target].mean():.4f}')

label: preprocess_v1_over50_train.csv      	nrows: 80224 	%target train: 0.5000 	%target val: 0.0852
label: preprocess_v1_smote20_train.csv     	nrows: 48134 	%target train: 0.1667 	%target val: 0.0852
label: preprocess_v1_smote50_train.csv     	nrows: 80224 	%target train: 0.5000 	%target val: 0.0852
label: preprocess_v1_smoteTomek20_train.csv 	nrows: 46412 	%target train: 0.1543 	%target val: 0.0852
label: preprocess_v1_smoteTomek50_train.csv 	nrows: 79638 	%target train: 0.5000 	%target val: 0.0852
label: preprocess_v1_train.csv             	nrows: 43846 	%target train: 0.0852 	%target val: 0.0852
label: preprocess_v1_under50_train.csv     	nrows: 7468 	%target train: 0.5000 	%target val: 0.0852
label: preprocess_v2_over50_train.csv      	nrows: 80224 	%target train: 0.5000 	%target val: 0.0852
label: preprocess_v2_smote20_train.csv     	nrows: 48134 	%target train: 0.1667 	%target val: 0.0852
label: preprocess_v2_smote50_train.csv     	nrows: 80224 	%target train: 0.5000 	%target v

In [13]:
train_file = 'preprocess_v1_train.csv'
val_file = 'preprocess_v1_val.csv'

In [14]:
from sklearn.model_selection import ParameterGrid

In [15]:
cv_grid = {'objective': ['reg:logistic'], 
           'eval_metric': ['auc'],
           'seed': [seed],
           'max_depth': [3, 6, 9],
           'min_child_weight': [5, 10, 25, 50],
           'gamma': [0, 2.5, 5]}

params_grid = list(ParameterGrid(cv_grid))

In [16]:
df_results = pd.DataFrame(columns = ['preproc_label', 'model_label', 'método', 'parámetros', 'columnas_out',
                                     'auc_train', 'auc_val', 'threshold','f1_train', 'f1_val'])


for xgb_params in tqdm(params_grid):
    
    for train_file in sorted(preproc_train):

        preproc_label = train_file.split('_train')[0]

        print('----------------------------------------------------------------------')
        print(xgb_params)
        print(train_file)
        print('----------------------------------------------------------------------')

        df_train = pd.read_csv(f'{PROCESSED}/{train_file}', compression = 'zip')
        df_val = pd.read_csv(f'{PROCESSED}/{preproc_val[0]}', compression = 'zip')

        X_train, y_train = df_train.drop(target, axis = 1), df_train[target]
        X_val, y_val = df_val.drop(target, axis = 1), df_val[target]

        dtrain =  xgb.DMatrix(data=X_train, label = y_train)
        dval  = xgb.DMatrix(data=X_val,   label = y_val)

        watch_list  = [(dtrain,'train'),(dval,'val')]

        xgb_fit = xgb.train(params = xgb_params, dtrain = dtrain, 
                            num_boost_round = 1000, early_stopping_rounds = 100, 
                            evals = watch_list, verbose_eval=False)
        
        xgb_params_export = xgb_params.copy()
        xgb_params_export.update(xgb_fit.attributes())

        probs_train = xgb_fit.predict(dtrain, ntree_limit = xgb_fit.best_iteration)
        probs_val = xgb_fit.predict(dval, ntree_limit = xgb_fit.best_iteration)

        auc_train = roc_auc_score(y_train, probs_train)
        auc_val = roc_auc_score(y_val, probs_val)

        #best threshold
        prec, recall, threshold = precision_recall_curve(y_train, probs_train)
        prec_recall = pd.DataFrame({'prec': prec[:-1], 'recall': recall[:-1], 'threshold': threshold})
        prec_recall['f1'] = 2*prec_recall['prec']*prec_recall['recall'] / (prec_recall['prec'] + prec_recall['recall'])
        prec_recall = prec_recall.sort_values(by = 'f1', ascending = False).head(1)

        #f1 scores
        best_threshold = prec_recall['threshold'].values[0]
        f1_train = prec_recall['f1'].values[0]

        labels_val = np.where(probs_val >= best_threshold, 1, 0)
        f1_val = f1_score(y_val, labels_val)

        print(f'auc_train: {auc_train:.6f} \tauc_val: {auc_val:.6f} \tf1_train: {f1_train:.6f} \tf1_val: {f1_val:.6f}')

        results = [preproc_label, 'xgboost_baseline', 'fit', xgb_params, '',
                  auc_train, auc_val, best_threshold, f1_train, f1_val]


        df_results.loc[len(df_results)] = results

  0%|          | 0/36 [00:00<?, ?it/s]

----------------------------------------------------------------------
{'eval_metric': 'auc', 'gamma': 0, 'max_depth': 3, 'min_child_weight': 5, 'objective': 'reg:logistic', 'seed': 2020}
preprocess_v1_over50_train.csv
----------------------------------------------------------------------
auc_train: 0.925740 	auc_val: 0.912398 	f1_train: 0.857260 	f1_val: 0.387935
----------------------------------------------------------------------
{'eval_metric': 'auc', 'gamma': 0, 'max_depth': 3, 'min_child_weight': 5, 'objective': 'reg:logistic', 'seed': 2020}
preprocess_v1_smote20_train.csv
----------------------------------------------------------------------
auc_train: 0.963750 	auc_val: 0.902189 	f1_train: 0.809268 	f1_val: 0.303494
----------------------------------------------------------------------
{'eval_metric': 'auc', 'gamma': 0, 'max_depth': 3, 'min_child_weight': 5, 'objective': 'reg:logistic', 'seed': 2020}
preprocess_v1_smote50_train.csv
---------------------------------------------

  3%|▎         | 1/36 [02:17<1:20:16, 137.61s/it]

auc_train: 0.940279 	auc_val: 0.910838 	f1_train: 0.867863 	f1_val: 0.389914
----------------------------------------------------------------------
{'eval_metric': 'auc', 'gamma': 0, 'max_depth': 3, 'min_child_weight': 10, 'objective': 'reg:logistic', 'seed': 2020}
preprocess_v1_over50_train.csv
----------------------------------------------------------------------
auc_train: 0.924304 	auc_val: 0.911213 	f1_train: 0.857913 	f1_val: 0.381606
----------------------------------------------------------------------
{'eval_metric': 'auc', 'gamma': 0, 'max_depth': 3, 'min_child_weight': 10, 'objective': 'reg:logistic', 'seed': 2020}
preprocess_v1_smote20_train.csv
----------------------------------------------------------------------
auc_train: 0.965741 	auc_val: 0.905190 	f1_train: 0.813068 	f1_val: 0.329000
----------------------------------------------------------------------
{'eval_metric': 'auc', 'gamma': 0, 'max_depth': 3, 'min_child_weight': 10, 'objective': 'reg:logistic', 'seed': 202

  6%|▌         | 2/36 [04:27<1:16:42, 135.38s/it]

...

auc_train: 0.921712 	auc_val: 0.905125 	f1_train: 0.848180 	f1_val: 0.373444
----------------------------------------------------------------------
{'eval_metric': 'auc', 'gamma': 5, 'max_depth': 9, 'min_child_weight': 25, 'objective': 'reg:logistic', 'seed': 2020}
preprocess_v1_over50_train.csv
----------------------------------------------------------------------
auc_train: 0.940970 	auc_val: 0.911430 	f1_train: 0.867336 	f1_val: 0.404523
----------------------------------------------------------------------
{'eval_metric': 'auc', 'gamma': 5, 'max_depth': 9, 'min_child_weight': 25, 'objective': 'reg:logistic', 'seed': 2020}
preprocess_v1_smote20_train.csv
----------------------------------------------------------------------
auc_train: 0.965295 	auc_val: 0.900184 	f1_train: 0.807353 	f1_val: 0.354293
----------------------------------------------------------------------
{'eval_metric': 'auc', 'gamma': 5, 'max_depth': 9, 'min_child_weight': 25, 'objective': 'reg:logistic', 'seed': 202

 97%|█████████▋| 35/36 [1:07:29<01:45, 105.31s/it]

auc_train: 0.915384 	auc_val: 0.903283 	f1_train: 0.844309 	f1_val: 0.365591
----------------------------------------------------------------------
{'eval_metric': 'auc', 'gamma': 5, 'max_depth': 9, 'min_child_weight': 50, 'objective': 'reg:logistic', 'seed': 2020}
preprocess_v1_over50_train.csv
----------------------------------------------------------------------
auc_train: 0.939347 	auc_val: 0.911762 	f1_train: 0.869260 	f1_val: 0.416577
----------------------------------------------------------------------
{'eval_metric': 'auc', 'gamma': 5, 'max_depth': 9, 'min_child_weight': 50, 'objective': 'reg:logistic', 'seed': 2020}
preprocess_v1_smote20_train.csv
----------------------------------------------------------------------
auc_train: 0.961805 	auc_val: 0.885820 	f1_train: 0.800281 	f1_val: 0.360307
----------------------------------------------------------------------
{'eval_metric': 'auc', 'gamma': 5, 'max_depth': 9, 'min_child_weight': 50, 'objective': 'reg:logistic', 'seed': 202

100%|██████████| 36/36 [1:09:28<00:00, 115.78s/it]

auc_train: 0.899683 	auc_val: 0.893918 	f1_train: 0.835035 	f1_val: 0.337800


In [19]:
df_results

,preproc_label,model_label,método,parámetros,columnas_out,auc_train,auc_val,threshold,f1_train,f1_val
0,preprocess_v1_over50,xgboost_baseline,fit,"{'eval_metric': 'auc', 'gamma': 0, 'max_depth'...",,0.925740,0.912398,0.519916,0.857260,0.387935
1,preprocess_v1_smote20,xgboost_baseline,fit,"{'eval_metric': 'auc', 'gamma': 0, 'max_depth'...",,0.963750,0.902189,0.363386,0.809268,0.303494
2,preprocess_v1_smote50,xgboost_baseline,fit,"{'eval_metric': 'auc', 'gamma': 0, 'max_depth'...",,0.994993,0.903298,0.397081,0.970218,0.316904
3,preprocess_v1_smoteTomek20,xgboost_baseline,fit,"{'eval_metric': 'auc', 'gamma': 0, 'max_depth'...",,0.984355,0.896496,0.335935,0.873732,0.352239
4,preprocess_v1_smoteTomek50,xgboost_baseline,fit,"{'eval_metric': 'auc', 'gamma': 0, 'max_depth'...",,0.993239,0.902251,0.425842,0.967290,0.307300
...,...,...,...,...,...,...,...,...,...,...
499,preprocess_v2_smote50,xgboost_baseline,fit,"{'eval_metric': 'auc', 'gamma': 5, 'max_depth'...",,0.993078,0.909270,0.449836,0.966119,0.519499
500,preprocess_v2_smoteTomek20,xgboost_baseline,fit,"{'eval_metric': 'auc', 'gamma': 5, 'max_depth'...",,0.972369,0.909576,0.357857,0.834015,0.543329
501,preprocess_v2_smoteTomek50,xgboost_baseline,fit,"{'eval_metric': 'auc', 'gamma': 5, 'max_depth'...",,0.993535,0.909469,0.429405,0.966711,0.522215
502,preprocess_v2,xgboost_baseline,fit,"{'eval_metric': 'auc', 'gamma': 5, 'max_depth'...",,0.917067,0.909057,0.256593,0.542299,0.531928


In [21]:
MODELS = DATA/'models'

In [22]:
df_results.to_csv(f'{MODELS}/xgboost_baseline.csv', index = False)

In [20]:
df_results.to_csv('resultados_xgboost_total.csv')